In [1]:
# Basic imports
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/cat_dog_data'


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize images to 224x224
    transforms.ToTensor(),           # Convert image to tensor
    transforms.Normalize([0.5], [0.5])  # Normalize
])


In [5]:
full_dataset = ImageFolder(data_dir, transform=transform)
print("Classes:", full_dataset.classes)
print("Total images:", len(full_dataset))


Classes: ['PetImages']
Total images: 24998


In [6]:
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)


In [7]:
resnet18 = models.resnet18(pretrained=True)
for param in resnet18.parameters():
    param.requires_grad = False  # Freeze all layers

resnet18.fc = nn.Linear(resnet18.fc.in_features, 1)  # 1 output for binary classification


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 52.5MB/s]


In [8]:
criterion = nn.BCEWithLogitsLoss()  # Best for binary classification
optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.001)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("✅ Using device:", device)

# Move model to the device (GPU if available)
resnet18 = resnet18.to(device)


✅ Using device: cpu


In [ ]:
from tqdm import tqdm  # Just once at the top of your notebook

for epoch in range(3):  # Train for 3 epochs
    resnet18.train()
    running_loss = 0.0

    # Add tqdm progress bar around the DataLoader
    loop = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch+1}")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Show current loss on progress bar
        loop.set_postfix(loss=loss.item())

    print(f"✅ Epoch {epoch+1} finished — Avg Loss: {running_loss/len(train_loader):.4f}")


Epoch 1:  94%|█████████▍| 586/625 [40:03<02:29,  3.83s/it, loss=7.59e-5]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1: 100%|██████████| 625/625 [42:51<00:00,  4.11s/it, loss=0.000427]


✅ Epoch 1 finished — Avg Loss: 0.0002


Epoch 2: 100%|██████████| 625/625 [02:27<00:00,  4.25it/s, loss=3.1e-5]


✅ Epoch 2 finished — Avg Loss: 0.0001


Epoch 3: 100%|██████████| 625/625 [02:28<00:00,  4.20it/s, loss=1.98e-5]

✅ Epoch 3 finished — Avg Loss: 0.0000


In [ ]:
model_path = '/content/drive/MyDrive/cat_vs_dog_resnet18.pth'
torch.save(resnet18.state_dict(), model_path)
print(f"Model saved to {model_path}")


Model saved to /content/drive/MyDrive/cat_vs_dog_resnet18.pth
